In [1]:
from pathlib import Path

import mne
from mne.datasets import sample, spm_face, testing
from mne.io import (
    read_raw_artemis123,
    read_raw_bti,
    read_raw_ctf,
    read_raw_fif,
    read_raw_kit,
)
from mne.viz import plot_alignment, set_3d_title

print(__doc__)

root_path = Path(mne.__file__).parent.absolute()

Automatically created module for IPython interactive environment


In [2]:
import matplotlib


In [3]:
%matplotlib widget

In [4]:
kwargs = dict(eeg=False, coord_frame="meg", show_axes=True, verbose=True)
raw = read_raw_ctf(
    spm_face.data_path() / "MEG" / "spm" / "SPM_CTF_MEG_example_faces1_3D.ds"
)
print(raw.info)

ds directory : /Users/hayato/mne_data/MNE-spm-face/MEG/spm/SPM_CTF_MEG_example_faces1_3D.ds
    res4 data read.
    hc data read.
    Separate EEG position data file not present.
    Quaternion matching (desired vs. transformed):
      -0.90   72.01    0.00 mm <->   -0.90   72.01   -0.00 mm (orig :  -43.09   61.46 -252.17 mm) diff =    0.000 mm
       0.90  -72.01    0.00 mm <->    0.90  -72.01   -0.00 mm (orig :   53.49  -45.24 -258.02 mm) diff =    0.000 mm
      98.30    0.00    0.00 mm <->   98.30   -0.00    0.00 mm (orig :   78.60   72.16 -241.87 mm) diff =    0.000 mm
    Coordinate transformations established.
    Polhemus data for 3 HPI coils added
    Device coordinate locations for 3 HPI coils added
    Measurement info composed.
Finding samples for /Users/hayato/mne_data/MNE-spm-face/MEG/spm/SPM_CTF_MEG_example_faces1_3D.ds/SPM_CTF_MEG_example_faces1_3D.meg4: 
    System clock channel is available, checking which samples are valid.
    1 x 324474 = 324474 samples from 340 ch

In [5]:
import numpy as np

pos = raw.info["chs"]

sensor_positions = []
for ch in pos:
    if ch["kind"] == mne.io.constants.FIFF.FIFFV_MEG_CH:
        sensor_positions.append(ch["loc"][:3])

sensor_positions = np.array(sensor_positions)

In [6]:
sensor_names = [
    ch["ch_name"] for ch in pos if ch["kind"] == mne.io.constants.FIFF.FIFFV_MEG_CH
]

In [7]:
print(sensor_names)
print(sensor_positions)
print(len(sensor_names))
print(len(sensor_positions))

['MLC11-2908', 'MLC12-2908', 'MLC13-2908', 'MLC14-2908', 'MLC15-2908', 'MLC16-2908', 'MLC17-2908', 'MLC21-2908', 'MLC22-2908', 'MLC23-2908', 'MLC24-2908', 'MLC25-2908', 'MLC31-2908', 'MLC32-2908', 'MLC41-2908', 'MLC42-2908', 'MLC51-2908', 'MLC52-2908', 'MLC53-2908', 'MLC54-2908', 'MLC55-2908', 'MLC61-2908', 'MLC62-2908', 'MLC63-2908', 'MLF11-2908', 'MLF12-2908', 'MLF13-2908', 'MLF14-2908', 'MLF21-2908', 'MLF22-2908', 'MLF23-2908', 'MLF24-2908', 'MLF25-2908', 'MLF31-2908', 'MLF32-2908', 'MLF33-2908', 'MLF34-2908', 'MLF35-2908', 'MLF41-2908', 'MLF42-2908', 'MLF43-2908', 'MLF44-2908', 'MLF45-2908', 'MLF46-2908', 'MLF51-2908', 'MLF52-2908', 'MLF53-2908', 'MLF54-2908', 'MLF55-2908', 'MLF56-2908', 'MLF61-2908', 'MLF62-2908', 'MLF63-2908', 'MLF64-2908', 'MLF65-2908', 'MLF66-2908', 'MLF67-2908', 'MLO11-2908', 'MLO12-2908', 'MLO13-2908', 'MLO14-2908', 'MLO21-2908', 'MLO22-2908', 'MLO23-2908', 'MLO24-2908', 'MLO31-2908', 'MLO32-2908', 'MLO33-2908', 'MLO34-2908', 'MLO41-2908', 'MLO43-2908', 'MLO4

In [8]:
excluded_indexes = [i for i, name in enumerate(sensor_names) if ("MLF25" in name or "MRF43" in name or "MRO13" in name or "MRO11" in name)]

name2position = dict(zip(sensor_names, sensor_positions))

print(excluded_indexes)

[32, 171, 188, 190]


In [9]:
valid_positions = [pos for i, pos in enumerate(sensor_positions) if i not in excluded_indexes]

In [10]:
print(len(valid_positions))

270


In [11]:
layout = mne.channels.find_layout(raw.info, ch_type="meg")

In [12]:
print(layout.pos.shape)

(275, 4)


In [13]:
print(layout.pos)

[[0.4528255  0.59689714 0.04486874 0.03365155]
 [0.4173133  0.57397155 0.04486874 0.03365155]
 [0.37753064 0.55778877 0.04486874 0.03365155]
 ...
 [0.47957203 0.09174734 0.04486874 0.03365155]
 [0.48013393 0.02825243 0.04486874 0.03365155]
 [0.47844822 0.23772945 0.04486874 0.03365155]]


In [14]:
print(sensor_names == sorted(sensor_names))

True


In [23]:
import torch

sensor_names = [
    ch["ch_name"][:5]
    for ch in raw.info["chs"]
    if ch["kind"] == mne.io.constants.FIFF.FIFFV_MEG_CH
] + ["MLO42"]  # なぜか上記のsensor_namesに含まれないセンサー名
# ["MLC11", "MLC12", ...]
excluded_sensor_names = ["MLF25", "MRF43", "MRO13", "MRO11"]

sensor_names = sorted(sensor_names)
sensor_names_in_data = [
    name for name in sensor_names if name not in excluded_sensor_names
]

print(len(sensor_names_in_data))
print(sensor_names_in_data.index("MLO42"))
layout = mne.channels.find_layout(raw.info, ch_type="meg")
positions = layout.pos[:, :2][
    [
        (name[:5] not in excluded_sensor_names)
        for name in sensor_names
    ]
]

torch_positions = torch.tensor(positions)
print(torch_positions.shape)
print(torch_positions)

271
69
torch.Size([271, 2])
tensor([[0.4528, 0.5969],
        [0.4173, 0.5740],
        [0.3775, 0.5578],
        [0.3370, 0.5340],
        [0.3027, 0.5067],
        [0.2747, 0.4719],
        [0.2482, 0.4307],
        [0.4123, 0.5323],
        [0.3731, 0.5158],
        [0.3395, 0.4900],
        [0.3143, 0.4572],
        [0.2893, 0.4194],
        [0.3578, 0.4518],
        [0.3288, 0.4135],
        [0.3912, 0.4774],
        [0.3694, 0.4118],
        [0.4534, 0.5258],
        [0.4309, 0.4895],
        [0.4000, 0.4368],
        [0.4053, 0.3945],
        [0.4246, 0.3594],
        [0.4537, 0.4545],
        [0.4364, 0.4184],
        [0.4568, 0.3830],
        [0.4130, 0.7905],
        [0.3536, 0.7778],
        [0.2949, 0.7598],
        [0.2365, 0.7401],
        [0.4466, 0.7605],
        [0.3952, 0.7516],
        [0.3460, 0.7374],
        [0.2930, 0.7162],
        [0.4250, 0.7231],
        [0.3780, 0.7128],
        [0.3284, 0.6933],
        [0.2852, 0.6717],
        [0.2411, 0.6457],
        [0

In [16]:
import sys

sys.path.append("../")

from lib.brain_module import SpatialAttention2D

model = SpatialAttention2D()
model.train()
X = torch.randn(2, 270, 281)
Y = model(X)

ModuleNotFoundError: No module named 'lib'

In [ ]:
from lib.brain_module import SubjectLayer

subject_layer = SubjectLayer()
Z = subject_layer(Y, 0)

f